# Data Science Principles and Practices (COMM054) Lab Week 5

Follow the instructions to complete each of these tasks. This set of exercises focusses on writing basic Python code to perform estimation using **SciPy**. Do not worry if you do not complete them all in the timetabled lab session.

This is not assessed but will help you gain practical experience for the module exam and coursework.

# Libraries

As in previous weeks, we will be using NumPy, SciPy, and Pyplot from Matplotlib, so import these first:

```python
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
```

In [1]:
import numpy as np
import scipy.stats as ss
#import matplotlib.pyplot as plt

# Estimation of population mean and population variance

Recall the estimators for the population mean and population variance discussed in lectures:

$$
\begin{align*}
\overline{x} &= \frac{1}{n}\sum_{i=1}^n x_i \qquad\qquad\text{sample mean}\\
s^2_\text{biased} &= \frac{1}{n}\sum_{i=1}^n (x_i-\overline{x})^2\qquad\text{biased sample variance}\\
s^2_\text{unbiased} &= \frac{1}{n-1}\sum_{i=1}^n (x_i-\overline{x})^2\qquad\text{unbiased sample variance}
\end{align*}
$$
where $n$ is the sample size. 

Let us first evaluate these by drawing 6 samples from a Poisson distribution (which you will recall has equal mean and variance):

```python
mu, n = 5, 6
x_samples = ss.poisson.rvs(mu, size=n)

mean = np.mean(x_samples)
var_b = np.var(x_samples)
var_ub = np.var(x_samples, ddof=1)
print("The true values of the mean and variance are", mu, "and", mu, "respectively")
print("The samples are", x_samples)
print("The sample mean is", mean)
print("The biased sample variance is", var_b)
print("The unbiased sample variance is", var_ub)
```

In [ ]:
mu, n = 5, 6
x_samples = ss.poisson.rvs(mu, size=n)

mean = np.mean(x_samples)
var_b = np.var(x_samples)
var_ub = np.var(x_samples, ddof=1)
print("The true values of the mean and variance are", mu, "and", mu, "respectively")
print("The samples are", x_samples)
print("The sample mean is", mean)
print("The biased sample variance is", var_b)
print("The unbiased sample variance is", var_ub)

Unbiased estimators have the property that their expectation equals the true value of the parameter. This means that if we consider a large number of experiments (in each case, taking a fixed number of samples $n$, and calculating the estimator on these samples), then the average of the estimates will tend to the true value of the parameter as the number of experiments increases. 

Let's see this in action for our case: we consider running 10000 experiments, where in each case we sample 6 values from the Poisson distribution. We calculate the sample mean, biased sample variance, and unbiased sample variance in each case, and average over the number of experiments:

```python
mu, n = 5, 6
num_experiments = 10000

mean_average, var_b_average, var_ub_average = 0, 0, 0

for i in range(num_experiments):
    x_samples = ss.poisson.rvs(mu, size=n)
    mean_average += np.mean(x_samples)             # sum all the sample means
    var_b_average += np.var(x_samples)             # sum all the biased sample variances
    var_ub_average += np.var(x_samples, ddof=1)    # sum all the unbiased sample variances

mean_average /= num_experiments   # average over all experiments
var_b_average /= num_experiments
var_ub_average /= num_experiments

print("The true values of the mean and variance are", mu, "and", mu, "respectively")
print("The average of the sample mean over " + str(num_experiments) + " experiments is", mean_average)
print("The average of the biased sample variance over " + str(num_experiments) + " experiments is", var_b_average)
print("The average of the unbiased sample variance over " + str(num_experiments) + " experiments is", var_ub_average)
```

In [ ]:
mu, n = 5, 6
num_experiments = 10000

mean_average, var_b_average, var_ub_average = 0, 0, 0

for i in range(num_experiments):
    x_samples = ss.poisson.rvs(mu, size=n)
    mean_average += np.mean(x_samples)             # sum all the sample means
    var_b_average += np.var(x_samples)             # sum all the biased sample variances
    var_ub_average += np.var(x_samples, ddof=1)    # sum all the unbiased sample variances

mean_average /= num_experiments   # average over all experiments
var_b_average /= num_experiments
var_ub_average /= num_experiments

print("The true values of the mean and variance are", mu, "and", mu, "respectively")
print("The average of the sample mean over " + str(num_experiments) + " experiments is", mean_average)
print("The average of the biased sample variance over " + str(num_experiments) + " experiments is", var_b_average)
print("The average of the unbiased sample variance over " + str(num_experiments) + " experiments is", var_ub_average)

You should see that the averages of the sample mean and unbiased sample variance closely agree with the true values of the mean and variance of the underlying distribution, while the average of the biased sample variance underestimates the true value of the variance (by a factor of approximately $\frac{n-1}{n}$).

Try playing around with some of the parameters in the program above to see what effect it has. Note that for a large number of samples $n$, the difference between the unbiased and biased sample variance becomes small (since $\frac{n-1}{n}\simeq 1$ for large $n$).

Repeat this example for a normal distribution with mean 5 and standard deviation 3:

In [ ]:
mu, sigma, n = 5, 3, 6
num_experiments = 10000

mean_average, var_b_average, var_ub_average = 0, 0, 0

for i in range(num_experiments):
    x_samples = ss.norm.rvs(loc=mu, scale=sigma, size=n)
    mean_average += np.mean(x_samples)             # sum all the sample means
    var_b_average += np.var(x_samples)             # sum all the biased sample variances
    var_ub_average += np.var(x_samples, ddof=1)    # sum all the unbiased sample variances

mean_average /= num_experiments   # average over all experiments
var_b_average /= num_experiments
var_ub_average /= num_experiments

print("The true values of the mean and variance are", mu, "and", sigma**2, "respectively")
print("The average of the sample mean over " + str(num_experiments) + " experiments is", mean_average)
print("The average of the biased sample variance over " + str(num_experiments) + " experiments is", var_b_average)
print("The average of the unbiased sample variance over " + str(num_experiments) + " experiments is", var_ub_average)

# Confidence intervals

## Normal distribution with unknown $\mu$, known $\sigma$

Suppose we sample a random variable $X\sim N(\mu,\sigma^2)$ that is distributed normally with mean $\mu$ and standard deviation $\sigma$, and obtain the sample $x_1, x_2,\ldots, x_n$. In lectures we showed that the (two-sided) confidence interval associated with this sample is 
$$
\left(\overline{x} - k\frac{\sigma}{\sqrt{n}},\ \overline{x}+k\frac{\sigma}{\sqrt{n}}\right)
$$
where $k$ takes on various values depending on the confidence percentage we are considering. Note that the endpoints of the CI are statistics, since we are assuming that $\sigma$ is **known**.
Some commonly considered values are

| CI% | $k$ |
|:---:|:---:|
| 90 | 1.645 |
| 95 | 1.96 |
| 98 | 2.326 |
| 99 | 2.576 |

(The general formula for $k$ for a given confidence percentage $C$ is
$
k = G\left(\frac{1}{2}\left(1+\frac{C}{100}\right)\right)
$,
where $G$ is the so-called *percent-point function* corresponding to the normal distribution. The percent-point function $G(\alpha)$ gives the value such that $P(Z<G(\alpha)) = \alpha$, where $Z\sim N(0,1)$. The percent-point function is implemented as `scipy.stats.norm.ppf` in SciPy.)

The code below samples a random variable $X\sim N(5, 3^2)$ 1000 times, and calculates the corresponding 95% confidence interval:

```python
mu, sigma, n = 5, 3, 1000
confidence_percentage = 95
k = ss.norm.ppf(1/2*(1+confidence_percentage/100))

x_samples = ss.norm.rvs(loc = mu, scale = sigma, size=n)

x_bar = np.mean(x_samples)
lower_bound, upper_bound = x_bar-k*sigma/np.sqrt(n), x_bar+k*sigma/np.sqrt(n)

print("The "+ str(confidence_percentage) + "% confidence interval for the sample is "+"(" + str(lower_bound) + "," + str(upper_bound) + ")")
```
Try this below:

In [ ]:
mu, sigma, n = 5, 3, 1000
confidence_percentage = 95
k = ss.norm.ppf(1/2*(1+confidence_percentage/100))

x_samples = ss.norm.rvs(loc = mu, scale = sigma, size=n)

x_bar = np.mean(x_samples)
lower_bound, upper_bound = x_bar-k*sigma/np.sqrt(n), x_bar+k*sigma/np.sqrt(n)

print("The "+ str(confidence_percentage) + "% confidence interval for the sample is "+"(" + str(lower_bound) + "," + str(upper_bound) + ")")

The confidence interval can also be calculated using the `interval()` method, which calculates the interval that contains 95% of the probability of a distribution. Since for the sample mean 
$$\overline{X}\sim N\left(\mu,\frac{\sigma^2}{n}\right),$$
we calculate the 95% confidence interval using:
```python
ss.norm.interval(0.95,np.mean(x_samples),sigma/np.sqrt(n))
```
Try this:

In [ ]:
print("Confidence interval generated using scipy.stats.norm.interval is", ss.norm.interval(0.95,np.mean(x_samples),sigma/np.sqrt(n)))

The interpretation of the 95% confidence interval is as follows: suppose we conduct a number of "experiments", where each experiment consists of sampling $n$ times from $X$, and calculating the corresponding confidence interval. Then for a large number of experiments, the true value of the (unknown to us) mean $\mu$ will lie within the confidence interval 95% of the time. Let's test this with the following code:
```python
mu, sigma, n = 5, 3, 1000  # n = number of samples taken in each experiment
confidence_percentage = 95  # 95% confidence interval
k = ss.norm.ppf(1/2*(1+confidence_percentage/100))

num_experiments = 100  
num_containing_mu = 0        # records the number of times our confidence interval contains the true value of the mean

for i in range(num_experiments):
    x_samples = ss.norm.rvs(loc=mu, scale=sigma, size=n)
    x_bar = np.mean(x_samples)
    lower_bound, upper_bound = x_bar-k*sigma/np.sqrt(n), x_bar+k*sigma/np.sqrt(n) # calculating the confidence interval
    if (mu >= lower_bound) and (mu <= upper_bound):
        num_containing_mu += 1       # if mu lies in the confidence interval, increment num_containing_mu by 1

print("The fraction of experiments where the confidence interval contains the true value of mu is", num_containing_mu/num_experiments)
```
Try this below. Experiment with adjusting `num_experiments`: as it gets larger, the fraction of times the CI contains $\mu$ should get closer to 95%. Also experiment with other values of `confidence_percentage`.

In [ ]:
mu, sigma, n = 5, 3, 1000  # n = number of samples taken in each experiment
confidence_percentage = 95  # 95% confidence interval
k = ss.norm.ppf(1/2*(1+confidence_percentage/100))

num_experiments = 100
num_containing_mu = 0        # records the number of times our confidence interval contains the true value of the mean

for i in range(num_experiments):
    x_samples = ss.norm.rvs(loc=mu, scale=sigma, size=n)
    x_bar = np.mean(x_samples)
    lower_bound, upper_bound = x_bar-k*sigma/np.sqrt(n), x_bar+k*sigma/np.sqrt(n) # calculating the confidence interval
    if (mu >= lower_bound) and (mu <= upper_bound):
        num_containing_mu += 1       # if mu lies in the confidence interval, increment num_containing_mu by 1

print("The fraction of experiments where the confidence interval contains the true value of mu is", num_containing_mu/num_experiments)

# Maximum likelihood estimators (MLEs)

## Normal distribution with unknown $\mu$, known $\sigma$ 

**scipy.stats** has a built-n `fit()` method that can calculate maximum likelihood estimates for its various distributions. We use it as follows:

```python
mu_true, sigma_true = 5, 2
x_samples = ss.norm.rvs(loc=mu_true, scale=sigma_true, size=1000)  # take 1000 samples

mu_MLE = ss.norm.fit(x_samples, fscale=sigma_true)[0]
print("The maximum likelihood estimate of the mean is", mu_MLE)

```

The `.fit()` method returns a list containing the MLE estimates for the given sample. In this example, we imagine we know the standard deviation $\sigma$, and are just interested in estimating the mean $\mu$. The optional argument `fscale=sigma_true` tells SciPy to fix the `scale` argument in the distribution (which corresponds to the standard deviation), so that maximises on the `loc` argument only (which corresponds to the mean).

Try this below:

In [ ]:
mu_true, sigma_true = 5, 2
x_samples = ss.norm.rvs(loc=mu_true, scale=sigma_true, size=1000) # take 1000 samples

mu_MLE = ss.norm.fit(x_samples, fscale=sigma_true)[0]
print("The maximum likelihood estimate of the mean is", mu_MLE)

We can compare this to the theoretical MLE estimator derived in the lectures (which turned out to just be the sample mean):
$$
\mu_\text{MLE}(x_1,\ldots,x_n) = \frac{1}{n}\sum_{i=1}x_i
$$
Calculate this value below:

In [ ]:
print(np.mean(x_samples))

You should see that the two values agree. Let's plot the histogram of the samples we drew, alongside the normal distribution with the given (known) standard deviation, and the MLE estimate for the mean:
```python
fig, ax = plt.subplots(nrows = 1, ncols=2)

ax[0].hist(x_samples, bins=50)
ax[0].set_title("histogram of samples")

x_range = np.linspace(min(x_samples),max(x_samples),100)
ax[1].plot(x_range, ss.norm.pdf(x_range, loc=mu_MLE, scale=sigma_true))
ax[1].set_title("pdf using mu_MLE")
ax[1].set_ylim(bottom=0)

plt.tight_layout()
plt.show()
```

The expression `np.linspace(min, max, num)` returns a list of `num` values starting from `min` and ending with `max`.

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols=2)

ax[0].hist(x_samples, bins=50)
ax[0].set_title("histogram of samples")

x_range = np.linspace(min(x_samples),max(x_samples),100)
ax[1].plot(x_range, ss.norm.pdf(x_range, loc=mu_MLE, scale=sigma_true))
ax[1].set_title("pdf using mu_MLE")
ax[1].set_ylim(bottom=0)

plt.tight_layout()
plt.show()

Using the `fit()` method works well since we are drawing from a known distribution. But let us suppose we did not know the theoretical expression for the MLE estimator (this might be the case, for example, if we were using our own custom probability distribution to model a problem). We would need to resort to numerically maximizing the likelihood function, or more simply, the log-likelihood. Recall the expression for this:
$$
\log L(\lambda_1,\ldots,\lambda_l) = \sum_{i=1}^n \log f(x_i;\lambda_1,\ldots,\lambda_l)
$$
where $x_1,\ldots, x_n$ is the sample, and $\lambda_1,\ldots,\lambda_l$ are the parameters our distribution function $f$. In the case of the normal distribution
$$
f(x;\mu,\sigma) = \frac{1}{\sqrt{2\pi}\sigma}\exp\left( -\frac{(x-\mu)^2}{2\sigma^2} \right)
$$
Since we are initially assuming $\sigma$ is known, we just consider the function 
$$
\log L(\mu) = \sum_{i=1}^n \log f(x_i;\mu,\sigma).
$$

We draw 1000 samples, and plot the log-likelihood using the following code:
```python
def loglikelihood(mu):
    value = 0
    for x in x_samples:
        value += np.log(ss.norm.pdf(x, loc=mu, scale=sigma_true))
    return value

mu_range = np.linspace(0,10,100)
plt.plot(mu_range, loglikelihood(mu_range))
plt.title("Plot of log-likelihood function")
plt.xlabel("mu")
plt.show()
```

In [ ]:
def loglikelihood(mu):
    value = 0
    for x in x_samples:
        value += np.log(ss.norm.pdf(x, loc=mu, scale=sigma_true))
    return value

mu_range = np.linspace(0,10,100)
plt.plot(mu_range, loglikelihood(mu_range))
plt.title("Plot of log-likelihood function")
plt.xlabel("mu")
plt.show()

Our goal is to find the value of $\mu$ that maximises the log-likelihood function. We can do this by using the `minimize` function from `scipy.optimize`. Maximizing $L(\mu)$ is equivalent to minimizing $-L(\mu)$, so we need to define the auxilliary function `minusloglikelihood`. Also, the minimize function requires an initial guess, which we can estimate from the plot above - we take 7 as our initial guess in the code below:

```python
import scipy.optimize as so

def minusloglikelihood(mu):
    return (-1)*loglikelihood(mu)

minimization_result = so.minimize(minusloglikelihood, 7)  # minimize with an initial guess of mu=7
print(minimization_result)
```

In [ ]:
import scipy.optimize as so

def minusloglikelihood(mu):
    return (-1)*loglikelihood(mu)

minimization_result = so.minimize(minusloglikelihood, 7)  # minimize with an initial guess of mu=7
print(minimization_result)

`minimization_result` contains a collection of useful information related to the minimization. The value of $\mu$ that actually minimizes $-\log L(\mu)$ is stored in `minimization_result.x` as a numpy array. From this, you can extract the MLE estimate for $\mu$:
```python
mu_MLE = minimization_result.x[0]
```
As above, try plotting the histogram of samples against the pdf generated using $\mu_\text{MLE}$ (and the true value of $\sigma$):

In [ ]:
mu_MLE = minimization_result.x[0]

fig, ax = plt.subplots(nrows = 1, ncols=2)

ax[0].hist(x_samples, bins=50)
ax[0].set_title("histogram of samples")

x_range = np.linspace(min(x_samples),max(x_samples),100)
ax[1].plot(x_range, ss.norm.pdf(x_range, loc=mu_MLE, scale=sigma_true))
ax[1].set_title("pdf using mu_MLE")
ax[1].set_ylim(bottom=0)

plt.tight_layout()
plt.show()